In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
np.set_printoptions(precision=3, suppress=True)
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import pandas as pd 
list_dict = {'Current_uA':time_points_r, 'E_field_V/cm':fv} 
df = pd.DataFrame(list_dict) 
df.to_csv('current_efield.csv', index=False)
print(df)

In [ ]:
from ROOT import TCanvas, TGraph, TFile, TH1F, TH2D
import numpy as np
import re
import scipy as sc
import matplotlib.pyplot as plt

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("ke", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[43:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    return pulsed_array, data_x, data_y

def collect_efield(file, position):
    field_values = []
    for f, p in zip(file, position):
        myFile = TFile.Open(f, "read")
        data = myFile.Get("ElectricFieldReader")
        detector = data.Get("detector1")
        efield = detector.Get("field1d_z")
        no_bins = efield.GetNbinsX()
        bin_size = 0.1/no_bins
        
        pos = p*10**(-3)
            
        placement = -0.05
        for bin in range(no_bins):  
            placement += bin_size
            if placement >= pos:
                field_strength = efield.GetBinContent(bin) 
                field_values.append(field_strength)
                break
                
    return field_values

def fit_convolve_data(x_data, y_data, time):
    time_points_r = []
    time_points_c = []
    
    for x_pos, y_pos in zip(x_data, y_data):
        if x_pos == [] and y_pos == []:
            time_points_r.append(0)
            time_points_c.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.1, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                for i in range(len(x)):
                    if x[i] >= 0.6 and x[i-1] < 0.6:
                        hole_index = i
                        break
                    else:
                        continue
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 0.35 and x_convolve[i-1] < 0.35:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                hole_cont = np.abs(y[hole_index])
                hole_cont_c = np.abs(y_convolve[hole_index_c])
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                    
                time_point_raw = np.abs(y[time_index_r])-hole_cont
                time_points_r.append(time_point_raw)
                
                time_point_con = np.abs(y_convolve[time_index_c])-hole_cont_c
                time_points_c.append(time_point_con)
        
    return time_points_r, time_points_c

def fit_data_less_overbias(x_data, y_data, time, p):
    time_points_r = []
    time_points_c = []
    
    for x_pos, y_pos, position in zip(x_data, y_data, p):

        if x_pos == [] and y_pos == []:
            time_points_r.append(0)
            time_points_c.append(0)

            
        else:
            for x, y in zip(x_pos, y_pos):  
                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.07, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                for i in range(len(x)):
                    if x[i] >= 4.5 and x[i-1] < 4.5:
                        hole_index = i
                        break
                    else:
                        continue
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 3 and x_convolve[i-1] < 3:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                hole_cont = np.abs(y[hole_index])
                hole_cont_c = np.abs(y_convolve[hole_index_c])
                
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                
                t_point_r = np.abs(y[time_index_r]) - hole_cont
                time_points_r.append(t_point_r)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                t_point_c = np.abs(y_convolve[time_index_c]) - hole_cont_c
                time_points_c.append(t_point_c)
        
    return time_points_r, time_points_c

files1 = ["data_10ps_n40.root", "extra_data_n35.root", "data_10ps_n30.root", "extra_data_n25.root","data_10ps_n20.root", "extra_data_n15.root", "data_10ps_n10.root", "extra_data_n5.root", "data_10ps_0.root"] 
files2 = ["less_overbias_n45.root", "less_overbias_n40.root", "less_overbias_n35.root", "less_overbias_n30.root", "less_overbias_n25.root", "less_overbias_n20.root", "less_overbias_n15.root", "less_overboas_n10.root", "less_overbias_n5.root", "less_overbias_0.root", "less_overbias_p5.root", "less_overbias_p10.root", "less_overbias_p15.root", "less_overbias_p20.root", "less_overbias_p25.root", "less_overbias_p30.root"]
position1 = [40, 35, 30, 25, 20, 15, 10, 5, 0] 
position2 = [45, 40, 35, 30, 25, 20, 15, 10, 5, 0, -5, -10, -15, -20, -25, -30]

x_values1, y_values1, pulse_names1 = [], [], []
for dat in files1:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names1.append(pulse)
    x_values1.append(x_dat)
    y_values1.append(y_dat)
    
x_values2, y_values2, pulse_names2 = [], [], []
for dat in files2:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names2.append(pulse)
    x_values2.append(x_dat)
    y_values2.append(y_dat)

fv = collect_efield(files1, position1) + collect_efield(files2, position2)
time_points_r1, time_points_c1 = fit_convolve_data(x_values1, y_values1, 0.1) 
time_points_r2, time_points_c2 = fit_data_less_overbias(x_values2, y_values2, 0.07, position2) 
time_points_r = time_points_r1+time_points_r2

for f, t in zip(fv, time_points_r):
    print(f,t)

In [ ]:
import pandas as pd 
list_dict = {'Current_uA':time_points_r, 'E_field_V/cm':np.abs(fv)} 
df = pd.DataFrame(list_dict) 
df.to_csv('current_efield.csv', index=False)
print(df)

In [ ]:
import tensorflow as tf
column_names = ['Current_uA', 'E_field_V/cm']

raw_dataset = pd.read_csv('current_efield.csv', names=column_names)
dataset = raw_dataset.copy()
dataset.tail()
dataset.isna().sum()
dataset = dataset.dropna()
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_dataset.describe().transpose()

train_dataset.describe().transpose()
train_features = train_dataset.copy()
test_features = test_dataset.copy()
test_features['Current_uA'].pop(0)

train_labels = train_features.pop('E_field_V/cm')
test_labels = test_features.pop('E_field_V/cm')

current = np.array(train_features, dtype="float32")
normalizer = layers.Normalization(input_shape=[1,], axis=None)
normalizer.adapt(current)

def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(1),
      layers.Dense(10000, activation='relu'),
      layers.Dense(10000, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

model = build_and_compile_model(normalizer)

train_features['Current_uA'] = train_features['Current_uA'].astype(float)
train_labels = train_labels.astype(float)

history = model.fit(
    train_features['Current_uA'],
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=1000)

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  #plt.ylim(0,1000)
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)
  
plot_loss(history)
print(history.history['loss'])

x = np.array([0.5])
test_features['Current_uA'] = test_features['Current_uA'].astype(float)
test_labels = test_labels.astype(float)
test_features['Current_uA'].pop(0)

y = model.predict(test_features['Current_uA'])
y1 = model.predict([x])
print(y1)
print(y)
test_results = {}
test_results['E_field_model'] = model.evaluate(
    test_features['Current_uA'], test_labels,
    verbose=0)

print(test_results)

In [ ]:
plt.scatter(train_features['Current_uA'], train_labels, label='Data')
plt.plot(test_features['Current_uA'], y, color='k', label='Predictions')
plt.xlabel('Current')
plt.ylabel('E_field')
plt.legend()